In [1]:
request_count = 0

In [4]:
import requests
import re
import json
from multiprocessing import Pool

# Function defined in another file to get it to work with multiprocessing
from get_game_metadata import get_game_metadata

with open('auth.json', 'r') as auth:
    auth_data = json.load(auth)
key = auth_data['key']

def get_friend_ids(steam_id):
    global request_count
    print('Sending request to get friendlist data [{0}]...'.format(request_count))
    request_count += 1
    
    res = json.loads(requests.get('http://api.steampowered.com/ISteamUser/GetFriendList/v0001/?key={0}&steamid={1}&relationship=friend'.format(key, steam_id)).text)
    
    friends = []
    if 'friendslist' in res:
        friendslist = res['friendslist']
        if 'friends' in friendslist:
            friends = friendslist['friends']
    
    friend_ids = []
    for friend in friends:
        friend_ids.append(friend['steamid'])
    return friend_ids

# Modifies complete_set input
def get_friend_traversal(start_id, levels, complete_set):
    complete_set.add(start_id)         
    current_list = get_friend_ids(start_id)
    
    while levels > 0:
        levels -= 1;
        temp_list = []
        for friend in current_list:
            if friend not in complete_set:
                complete_set.add(friend)
                if levels > 0:
                    new_ids = get_friend_ids(friend)
                    if len(new_ids) > 0:
                        temp_list.extend(new_ids)
        current_list = temp_list.copy()
    
def get_games(steam_id):
    res = json.loads(requests.get('http://api.steampowered.com/IPlayerService/GetOwnedGames/v0001/?key={0}&steamid={1}&format=json&include_appinfo=true&include_played_free_games=true'.format(key, steam_id)).text)
    res = res['response']
    games = []
    if 'games' in res:
        games = res['games']
    return games

In [3]:
games

NameError: name 'games' is not defined

In [3]:
# Get User IDs (already retrieved ~150,000 users, check Google Drive)

# complete_set = set([])
# get_friend_traversal('76561198023414915', 2, complete_set)
# get_friend_traversal('76561198048033045', 2, complete_set)
# get_friend_traversal('76561198163038211', 2, complete_set)

# steam_ids = {'steam_ids': complete_set}
# with open('steam_ids.json', 'w') as out:
#     json.dump(steam_ids, out)
# print('steam_ids.json file saved with {1} entries!'.format(len(friends)))

In [4]:
# Get All Steam Games (very quick run time)

# with open('all_games_basic.json', 'w') as out:
#     all_games = json.loads(requests.get('https://api.steampowered.com/ISteamApps/GetAppList/v2/').text)
#     json.dump(json.loads(requests.get('https://api.steampowered.com/ISteamApps/GetAppList/v2/').text), out)
#     print('all_games_basic.json saved with {0} entries'.format(len(all_games)))

In [5]:
# Get Metadata for All Games (will take ~19 hours but provides data for ALL games currently on the Steam store)

# with open('all_games_basic.json', 'r') as file:
#     all_games = json.load(file)['applist']['apps']

# updated_games = {}
# count = 1
# for game in all_games:
#     if count > 100:
#         break
#     print('Processing games [{0} / {1}]'.format(count, len(all_games)))
#     rating, tags, dlc = get_game_metadata(game['appid'])
#     if not dlc:
#         updated_games[game['appid']] = {'name': game['name'], 'rating': rating, 'tags': tags}
#     count += 1

# with open('all_games.json', 'w') as out:
#     json.dump(updated_games, out)
#     print('all_games.json saved with {0} entries'.format(len(updated_games)))

In [15]:
import datetime

with open('steam_ids.json', 'r') as file:
    steam_ids = json.load(file)['steam_ids']

all_games_from_users = {}
users_with_games = {}
count = 1
MAX_USERS = 1000

print_lines = 1
for steam_id in steam_ids:
    if count > MAX_USERS:
        break
    print('Getting games for account [{0} / {1}]'.format(count, MAX_USERS), end='\r')
    current_games = get_games(steam_id)
    if len(current_games) > 0:
        users_with_games[steam_id] = {}
        new_games = []
        urls = []
        for game in current_games:
            users_with_games[steam_id][game['appid']] = {'name': game['name'], 'playtime': game['playtime_forever']}
            if game['appid'] not in all_games_from_users:
                new_games.append(game)
                urls.append('https://store.steampowered.com/app/{0}'.format(game['appid']))
        
        with Pool(15) as p:
            records = p.map(get_game_metadata, urls)

        for x in range(len(new_games)):
            if records[x]:
                rating, tags, dlc = records[x]
                if not dlc:
                    all_games_from_users[new_games[x]['appid']] = {'name': new_games[x]['name'], 'rating': rating, 'tags': tags}
    count += 1

with open('all_games_from_users.json', 'w') as out:
    json.dump(all_games_from_users, out)
    print('all_games_from_users.json saved with {0} entries'.format(len(all_games_from_users)))

with open('users_with_games.json', 'w') as out:
    json.dump(users_with_games, out)
    print('users_with_games.json saved with {0} entries'.format(len(users_with_games)))

users_with_games.json saved with 417 entries
